In [1]:
import pandas as pd
import numpy as np
import branca.colormap as cm
import folium
import geopandas as gpd
import datetime
from folium.plugins import TimeSliderChoropleth

In [2]:
## load data
death = pd.read_csv('tidytuesday/data/2018/2018-11-13/malaria_deaths.csv')
death_age = pd.read_csv('tidytuesday/data/2018/2018-11-13/malaria_deaths_age.csv')
inc = pd.read_csv('tidytuesday/data/2018/2018-11-13/malaria_inc.csv')

In [3]:
## rename death_rate
death.columns = ['death_rate' if x=='Deaths - Malaria - Sex: Both - Age: Age-standardized (Rate) (per 100,000 people)' else x for x in death.columns]

In [4]:
sorted_df = death.sort_values(['Entity', 
                     'Year']).reset_index(drop=True)
sorted_df['date'] = sorted_df['Year'].apply(lambda x: datetime.datetime(x+1, 1, 1))
sorted_df['date'] = pd.to_datetime(sorted_df['date'], yearfirst=True).astype(int) / 10**9
sorted_df['date'] = sorted_df['date'].astype(int)

In [5]:
countries = gpd.read_file('Countries_WGS84.shp')
countries = countries.rename(columns={'CNTRY_NAME': 'Entity'})

In [6]:
joined_df = sorted_df.merge(countries, on='Entity')

In [7]:
joined_df = joined_df[['Entity', 'death_rate', 'date', 'geometry']]

In [8]:
mymap = folium.Map(tiles='cartodbpositron')
mymap.save(outfile='infinite_scroll.html')

In [9]:
mymap_fix_boundary = folium.Map(min_zoom=2, max_bounds=True, tiles='cartodbpositron')
mymap_fix_boundary.save(outfile='fix_boundary.html')

In [10]:
max_colour = max(joined_df['death_rate'])
min_colour = min(joined_df['death_rate'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
joined_df['colour'] = joined_df['death_rate'].map(cmap)

In [11]:
country_list = joined_df['Entity'].unique().tolist()
country_idx = range(len(country_list))

style_dict = {}
for i in country_idx:
    country = country_list[i]
    result = joined_df[joined_df['Entity'] == country]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict


In [12]:
countries_df = joined_df[['geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_gdf = countries_gdf.drop_duplicates().reset_index()

In [13]:
slider_map = folium.Map(min_zoom=2, max_bounds=True, tiles='cartodbpositron')
               
_ = TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Malaria Deaths (per 100,000 people)- Sex: Both - Age: Age-standardized"
slider_map.save(outfile='malaria_deaths.html')

Now you can go to [malaria_deaths.html](malaria_deaths.html) to see the plot.